# Part 2.5: 샘플 논문 준비하기

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. AI 리뷰를 위한 **논문 원고 준비** 방법 이해
2. 본인 논문 상태에 맞는 **샘플 논문 생성**
3. 각 섹션별 내용을 지정하여 **맞춤형 논문** 작성

---

## 논문 준비 옵션

| 옵션 | 현재 상태 | 생성 내용 |
|------|----------|----------|
| **A. 본인 논문** | 완성된 초고 있음 | 변환만 (PDF/DOCX → MD) |
| **B. Method만** | Method/표만 있음 | Intro + Results + Discussion 추가 |
| **C. 논문 없음** | 아무것도 없음 | 전체 논문 생성 (영어) |

---
## Part 1: Setup

In [ ]:
# Cell 1: 환경 설정
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경에서 실행 중")
except ImportError:
    current_dir = os.getcwd()
    if current_dir.endswith('notebooks'):
        os.chdir('..')
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

# input 폴더 생성
INPUT_DIR = os.path.join(WORKSHOP_DIR, "input")
os.makedirs(INPUT_DIR, exist_ok=True)

print(f"작업 폴더: {WORKSHOP_DIR}")
print(f"논문 저장 위치: {INPUT_DIR}")

In [ ]:
# Cell 2: API Key 로딩 및 의존성 설치
import os

# Colab에서 필요한 패키지 설치
if IN_COLAB:
    %pip install langchain-google-genai pdfplumber python-docx -q
    print("✅ 패키지 설치 완료")

if IN_COLAB:
    try:
        from google.colab import userdata
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    except:
        GEMINI_API_KEY = None
else:
    from dotenv import load_dotenv
    load_dotenv(override=True)
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

if GEMINI_API_KEY and not GEMINI_API_KEY.startswith('your_'):
    os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
    print("✅ Gemini API Key 로드 완료")
else:
    print("❌ Gemini API Key가 필요합니다 (옵션 B, C 사용 시)")
    print("   발급: https://aistudio.google.com/apikey")

---
## 옵션 A: 본인 논문 변환

이미 완성된 논문 초고가 있을 때 사용합니다. PDF나 Word 파일을 Markdown으로 변환합니다.

In [ ]:
# Cell 3: 본인 논문 변환 (PDF/DOCX → Markdown)

# ✏️ 변환할 파일 경로 입력
MY_FILE_PATH = ""  # 예: "input/my_paper.pdf" 또는 "input/my_paper.docx"

# ─────────────────────────────────────────

if not MY_FILE_PATH:
    print("ℹ️ MY_FILE_PATH에 파일 경로를 입력하세요.")
    print()
    print("예시:")
    print('   MY_FILE_PATH = "input/my_paper.pdf"')
    print('   MY_FILE_PATH = "input/my_paper.docx"')
    print('   MY_FILE_PATH = "/Users/name/Documents/thesis.pdf"')
elif not os.path.exists(MY_FILE_PATH):
    print(f"❌ 파일을 찾을 수 없습니다: {MY_FILE_PATH}")
else:
    ext = os.path.splitext(MY_FILE_PATH)[1].lower()
    text_content = ""
    
    if ext == '.pdf':
        try:
            import pdfplumber
            with pdfplumber.open(MY_FILE_PATH) as pdf:
                for page in pdf.pages:
                    text_content += page.extract_text() or ""
                    text_content += "\n\n"
            print(f"✅ PDF 추출 완료: {len(text_content)} 문자")
        except ImportError:
            print("⚠️ pdfplumber 필요: pip install pdfplumber")
            
    elif ext in ['.docx', '.doc']:
        try:
            import docx
            doc = docx.Document(MY_FILE_PATH)
            text_content = "\n\n".join([para.text for para in doc.paragraphs if para.text.strip()])
            print(f"✅ Word 추출 완료: {len(text_content)} 문자")
        except ImportError:
            print("⚠️ python-docx 필요: pip install python-docx")
            
    elif ext in ['.md', '.txt']:
        with open(MY_FILE_PATH, 'r', encoding='utf-8') as f:
            text_content = f.read()
        print(f"✅ 텍스트 로드 완료: {len(text_content)} 문자")
    else:
        print(f"⚠️ 지원하지 않는 파일 형식: {ext}")
        print("   지원 형식: .pdf, .docx, .doc, .md, .txt")
    
    # my_manuscript.md로 저장
    if text_content:
        output_path = os.path.join(INPUT_DIR, "my_manuscript.md")
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(text_content)
        
        print(f"\n💾 저장 완료: {output_path}")
        print(f"\n📄 미리보기 (처음 500자):")
        print("="*50)
        print(text_content[:500])
        print("\n✅ 노트북 3, 4에서 이 논문이 자동으로 사용됩니다!")

---
## 옵션 B: Method만 있는 경우

Methods 섹션(표, 분석 계획 등)은 있지만 다른 섹션이 없을 때 사용합니다.  
기존 Methods를 바탕으로 Introduction, Results, Discussion을 생성합니다.

> 💡 **Tip**: Methods가 없으면 샘플(기후불안 연구)이 자동 로드됩니다.

In [ ]:
# Cell 4: 옵션 B - Method 입력 및 섹션별 내용 지정

# ✏️ Method 파일 경로 (권장) 또는 직접 붙여넣기
MY_METHODS_PATH = ""  # 예: "input/my_methods.md" 또는 "/Users/name/Documents/methods.docx"

# 또는 직접 붙여넣기 (파일이 없을 때)
MY_METHODS = """

"""  # 여기에 기존 Methods 내용을 붙여넣으세요

# ✏️ 각 섹션에 포함할 내용 (키워드나 간단한 설명, 선택사항)
INTRO_CONTENT = ""  # 예: "climate anxiety, regional differences, longitudinal design"
RESULTS_CONTENT = ""  # 예: "significant trajectory differences, coastal > urban, income moderation"
DISCUSSION_CONTENT = ""  # 예: "targeted interventions, vulnerable populations, seasonal patterns"

# ─────────────────────────────────────────

# 1순위: 파일 경로에서 로드
if MY_METHODS_PATH:
    if os.path.exists(MY_METHODS_PATH):
        ext = os.path.splitext(MY_METHODS_PATH)[1].lower()
        
        if ext == '.pdf':
            try:
                import pdfplumber
                with pdfplumber.open(MY_METHODS_PATH) as pdf:
                    MY_METHODS = "\n\n".join([page.extract_text() or "" for page in pdf.pages])
                print(f"✅ PDF에서 Method 로드: {MY_METHODS_PATH}")
            except ImportError:
                print("⚠️ pdfplumber 필요: pip install pdfplumber")
        elif ext in ['.docx', '.doc']:
            try:
                import docx
                doc = docx.Document(MY_METHODS_PATH)
                MY_METHODS = "\n\n".join([para.text for para in doc.paragraphs if para.text.strip()])
                print(f"✅ Word에서 Method 로드: {MY_METHODS_PATH}")
            except ImportError:
                print("⚠️ python-docx 필요: pip install python-docx")
        elif ext in ['.md', '.txt']:
            with open(MY_METHODS_PATH, 'r', encoding='utf-8') as f:
                MY_METHODS = f.read()
            print(f"✅ 텍스트에서 Method 로드: {MY_METHODS_PATH}")
        else:
            print(f"⚠️ 지원하지 않는 파일 형식: {ext}")
    else:
        print(f"❌ 파일을 찾을 수 없습니다: {MY_METHODS_PATH}")

# 2순위: 샘플 Method 자동 로드 (둘 다 비어있을 때)
if not MY_METHODS.strip():
    SAMPLE_METHOD_PATH = os.path.join(INPUT_DIR, "sample_method.md")
    if os.path.exists(SAMPLE_METHOD_PATH):
        with open(SAMPLE_METHOD_PATH, 'r', encoding='utf-8') as f:
            MY_METHODS = f.read()
        print("📂 샘플 Method 자동 로드: sample_method.md")
        print("   (기후불안 청소년 종단연구)")
    else:
        print("⚠️ Method가 없습니다.")
        print("   MY_METHODS_PATH에 파일 경로를 입력하거나")
        print("   MY_METHODS에 직접 붙여넣으세요.")

# 결과 출력
if MY_METHODS.strip():
    print(f"\n📝 Method 입력 확인:")
    print(f"   - Method 길이: {len(MY_METHODS)} 문자")
    print(f"   - Introduction 힌트: {INTRO_CONTENT or '(자동 생성)'}")
    print(f"   - Results 힌트: {RESULTS_CONTENT or '(자동 생성)'}")
    print(f"   - Discussion 힌트: {DISCUSSION_CONTENT or '(자동 생성)'}")
    print("\n✅ 다음 셀을 실행하여 논문을 완성하세요.")

In [ ]:
# Cell 5: 옵션 B - Method 기반 논문 완성 및 저장
from langchain_google_genai import ChatGoogleGenerativeAI

if 'MY_METHODS' not in dir() or not MY_METHODS.strip():
    print("❌ 위 셀(Cell 4)을 먼저 실행하세요.")
else:
    print("🔄 Method를 바탕으로 논문 완성 중...")
    
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
    
    prompt = f"""You are a psychology researcher. I have a Methods section already written. 
Please complete the paper by writing the other sections that match this methodology.

## EXISTING METHODS (use this as the style and content guide)
{MY_METHODS[:4000]}

---

## CONTENT HINTS FROM USER
- Introduction should cover: {INTRO_CONTENT or 'relevant background and research gaps based on the methods'}
- Results should include: {RESULTS_CONTENT or 'statistical findings consistent with the methods'}
- Discussion should address: {DISCUSSION_CONTENT or 'interpretation, implications, and limitations'}

---

## YOUR TASK
Write a complete academic paper in **English** with these sections:

1. **Title** - specific and academic
2. **Abstract** (200-250 words) - background, purpose, methods, results, conclusion
3. **Introduction** (800-1000 words)
   - Research background and importance
   - Literature review (cite 5-8 hypothetical studies)
   - Research gap and hypotheses
4. **Methods** - USE THE PROVIDED METHODS ABOVE (copy it exactly, just format as Markdown)
5. **Results** (500-700 words)
   - Statistical findings matching the methods
   - Include: M, SD, t/F values, p values, effect sizes (d, r, η²)
   - Reference to tables/figures
6. **Discussion** (800-1000 words)
   - Start with BIG PICTURE (not result listing)
   - Interpretation with prior literature
   - Theoretical and practical implications
   - Limitations and future directions
7. **References** (APA 7th, 8-12 references)

## Requirements
- Academic tone throughout
- Results must be consistent with the provided Methods
- Use Markdown format (# Title, ## Section, **bold**, etc.)
- Maintain coherence between all sections

Write the complete paper:
"""
    
    response = llm.invoke(prompt)
    generated_paper = response.content
    
    # 자동 저장
    output_path = os.path.join(INPUT_DIR, "my_manuscript.md")
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(generated_paper)
    
    print(f"\n✅ 논문 완성 및 저장! ({len(generated_paper)} 문자)")
    print(f"💾 저장 위치: {output_path}")
    print("\n" + "="*60)
    print("📄 생성된 논문 미리보기:")
    print("="*60)
    print(generated_paper[:1500])
    print("\n...")
    print("\n✅ 노트북 3, 4에서 이 논문이 자동으로 사용됩니다!")

---
## 옵션 C: 논문이 전혀 없는 경우

**아무 초고도 없는 경우** 사용합니다. 각 섹션에 포함할 내용을 간단히 입력하면 전체 논문을 생성합니다.

> **Language**: 영어 (English)로 생성됩니다.

In [ ]:
# Cell 7: 옵션 C - 섹션별 내용 지정

# ✏️ 각 섹션에 포함할 내용 입력 (키워드, 문장, 또는 간단한 설명)

# Introduction에 들어갈 내용
C_INTRO = ""  
# 예: "relationship between social media use and depression in adolescents, 
#      prior studies show mixed results, need for longitudinal design"

# Methods에 들어갈 내용
C_METHODS = ""  
# 예: "N=300 high school students, 6-month longitudinal, PHQ-9, social media use questionnaire,
#      multilevel modeling"

# Results에 들어갈 내용
C_RESULTS = ""  
# 예: "significant positive correlation, gender moderation effect, 
#      passive use more harmful than active use"

# Discussion에 들어갈 내용
C_DISCUSSION = ""  
# 예: "implications for digital literacy education, parental involvement,
#      limitation: self-report measures, future: experimental design"

# ─────────────────────────────────────────

has_content = any([C_INTRO, C_METHODS, C_RESULTS, C_DISCUSSION])

if has_content:
    print("📝 섹션별 내용 확인:")
    print(f"   - Introduction: {C_INTRO[:80] + '...' if len(C_INTRO) > 80 else C_INTRO or '(없음)'}")
    print(f"   - Methods: {C_METHODS[:80] + '...' if len(C_METHODS) > 80 else C_METHODS or '(없음)'}")
    print(f"   - Results: {C_RESULTS[:80] + '...' if len(C_RESULTS) > 80 else C_RESULTS or '(없음)'}")
    print(f"   - Discussion: {C_DISCUSSION[:80] + '...' if len(C_DISCUSSION) > 80 else C_DISCUSSION or '(없음)'}")
    print("\n✅ 다음 셀을 실행하여 논문을 생성하세요.")
else:
    print("⚠️ 각 섹션에 포함할 내용을 입력하세요.")
    print("\n예시:")
    print('C_INTRO = "social media and adolescent depression, screen time effects"')
    print('C_METHODS = "N=300 students, 6-month longitudinal, PHQ-9, multilevel modeling"')
    print('C_RESULTS = "significant correlation r=0.35, gender moderation"')
    print('C_DISCUSSION = "clinical implications, digital literacy education"')

In [ ]:
# Cell 8: 옵션 C - Full 논문 생성
from langchain_google_genai import ChatGoogleGenerativeAI

has_content = any([C_INTRO, C_METHODS, C_RESULTS, C_DISCUSSION])

if not has_content:
    print("❌ 위 셀에서 섹션별 내용을 먼저 입력하세요.")
else:
    print("🔄 입력 내용을 바탕으로 논문 생성 중...")
    
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
    
    prompt = f"""You are a psychology researcher. Write a complete academic paper based on the following content specifications.

## USER'S CONTENT SPECIFICATIONS

**Introduction should cover:**
{C_INTRO or 'General psychology research topic - you may choose a relevant topic'}

**Methods should include:**
{C_METHODS or 'Standard quantitative methodology with appropriate sample and measures'}

**Results should show:**
{C_RESULTS or 'Significant findings that support the hypotheses'}

**Discussion should address:**
{C_DISCUSSION or 'Interpretation, implications, limitations, and future directions'}

---

## YOUR TASK
Write a complete academic paper in **English** with these sections:

1. **Title** - specific and academic, reflecting the content above

2. **Abstract** (200-250 words)
   - Background, purpose, methods, results, conclusion

3. **Introduction** (800-1000 words)
   - Research background and importance
   - Literature review with 5-8 hypothetical citations
   - Research gap identification
   - Clear research questions and hypotheses

4. **Methods** (600-800 words)
   - Participants: sample size, demographics, recruitment
   - Measures: scales with reliability coefficients
   - Procedure: data collection process
   - Data Analysis: statistical approach

5. **Results** (500-700 words)
   - Descriptive statistics
   - Main analyses with specific statistics (M, SD, t, F, p, d, r, η²)
   - Tables/figures references

6. **Discussion** (800-1000 words)
   - Start with BIG PICTURE meaning (not result listing!)
   - Interpretation in context of prior research
   - Theoretical and practical implications
   - Limitations
   - Future research directions
   - Conclusion

7. **References** (APA 7th format, 10-15 references)

## Requirements
- Academic tone throughout
- All sections must be coherent and consistent
- Use Markdown format (# Title, ## Section, **bold**, etc.)
- Include specific hypothetical data that looks realistic

Write the complete paper:
"""
    
    response = llm.invoke(prompt)
    generated_paper = response.content
    
    print(f"\n✅ 논문 생성 완료! ({len(generated_paper)} 문자)")
    print("\n" + "="*60)
    print("📄 생성된 논문 미리보기:")
    print("="*60)
    print(generated_paper[:1500])
    print("\n...")

In [ ]:
# Cell 7: 옵션 C - Full 논문 생성 및 저장
from langchain_google_genai import ChatGoogleGenerativeAI

if 'C_INTRO' not in dir():
    print("❌ 위 셀(Cell 6)을 먼저 실행하세요.")
else:
    has_content = any([C_INTRO, C_METHODS, C_RESULTS, C_DISCUSSION])
    
    if not has_content:
        print("❌ 위 셀에서 섹션별 내용을 먼저 입력하세요.")
    else:
        print("🔄 입력 내용을 바탕으로 논문 생성 중...")
        
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
        
        prompt = f"""You are a psychology researcher. Write a complete academic paper based on the following content specifications.

## USER'S CONTENT SPECIFICATIONS

**Introduction should cover:**
{C_INTRO or 'General psychology research topic - you may choose a relevant topic'}

**Methods should include:**
{C_METHODS or 'Standard quantitative methodology with appropriate sample and measures'}

**Results should show:**
{C_RESULTS or 'Significant findings that support the hypotheses'}

**Discussion should address:**
{C_DISCUSSION or 'Interpretation, implications, limitations, and future directions'}

---

## YOUR TASK
Write a complete academic paper in **English** with these sections:

1. **Title** - specific and academic, reflecting the content above

2. **Abstract** (200-250 words)
   - Background, purpose, methods, results, conclusion

3. **Introduction** (800-1000 words)
   - Research background and importance
   - Literature review with 5-8 hypothetical citations
   - Research gap identification
   - Clear research questions and hypotheses

4. **Methods** (600-800 words)
   - Participants: sample size, demographics, recruitment
   - Measures: scales with reliability coefficients
   - Procedure: data collection process
   - Data Analysis: statistical approach

5. **Results** (500-700 words)
   - Descriptive statistics
   - Main analyses with specific statistics (M, SD, t, F, p, d, r, η²)
   - Tables/figures references

6. **Discussion** (800-1000 words)
   - Start with BIG PICTURE meaning (not result listing!)
   - Interpretation in context of prior research
   - Theoretical and practical implications
   - Limitations
   - Future research directions
   - Conclusion

7. **References** (APA 7th format, 10-15 references)

## Requirements
- Academic tone throughout
- All sections must be coherent and consistent
- Use Markdown format (# Title, ## Section, **bold**, etc.)
- Include specific hypothetical data that looks realistic

Write the complete paper:
"""
        
        response = llm.invoke(prompt)
        generated_paper = response.content
        
        # 자동 저장
        output_path = os.path.join(INPUT_DIR, "my_manuscript.md")
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(generated_paper)
        
        print(f"\n✅ 논문 생성 및 저장! ({len(generated_paper)} 문자)")
        print(f"💾 저장 위치: {output_path}")
        print("\n" + "="*60)
        print("📄 생성된 논문 미리보기:")
        print("="*60)
        print(generated_paper[:1500])
        print("\n...")
        print("\n✅ 노트북 3, 4에서 이 논문이 자동으로 사용됩니다!")

In [ ]:
# Cell 10: 최종 확인

print("="*60)
print("📋 논문 준비 상태")
print("="*60)

my_path = os.path.join(INPUT_DIR, "my_manuscript.md")
sample_path = os.path.join(INPUT_DIR, "sample_manuscript.md")

print("\n📄 논문 로딩 우선순위 (노트북 3, 4):")

if os.path.exists(my_path):
    size = os.path.getsize(my_path) / 1024
    print(f"   1순위: ✅ my_manuscript.md ({size:.1f} KB) ← 사용됨")
else:
    print(f"   1순위: ❌ my_manuscript.md (없음)")

if os.path.exists(sample_path):
    size = os.path.getsize(sample_path) / 1024
    if os.path.exists(my_path):
        print(f"   2순위: ✅ sample_manuscript.md ({size:.1f} KB)")
    else:
        print(f"   2순위: ✅ sample_manuscript.md ({size:.1f} KB) ← 사용됨")
else:
    print(f"   2순위: ❌ sample_manuscript.md (없음)")

# 결과
if os.path.exists(my_path) or os.path.exists(sample_path):
    print("\n🎉 다음 단계: 노트북 3 또는 4를 실행하세요!")
else:
    print("\n❌ 논문 파일이 없습니다.")
    print("   위의 옵션 A, B, 또는 C를 실행하세요.")

---
## 다음 단계

| 노트북 | 내용 | 필요한 파일 |
|--------|------|-------------|
| **3. AI Paper Review Agent** | agentic-paper-review로 리뷰 받기 | `input/my_manuscript.md` |
| **4. Transparent Peer Review** | Few-shot vs Agentic 비교 | `input/my_manuscript.md` + 노트북 3 결과 |

### 옵션 요약

| 옵션 | 입력 | 출력 |
|------|------|------|
| **A** | 완성된 논문 (PDF/DOCX) | `my_manuscript.md` |
| **B** | Methods + 섹션별 힌트 | `my_manuscript.md` (Method가 few-shot 역할) |
| **C** | 섹션별 내용 지정 | `my_manuscript.md` (영어 전체 논문) |